In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2

In [2]:
spg.__version__

'1.2.2'

In [1]:
!pwd

/data/work/previous/ScAT_github/tutorial


In [3]:
#Read in gene expression and spatial location
adata=sc.read("../data/GF_spatial.h5ad")
adata

AnnData object with n_obs × n_vars = 34092 × 22747
    obs: 'x', 'y'
    uns: 'sample_info'
    obsm: 'spatial'

In [11]:
adata.obs["x_pixel"] = adata.obsm['spatial'][:, 0] - 0
adata.obs["y_pixel"] = adata.obsm['spatial'][:, 1] - 0
adata.obs['x_array'] = adata.obsm['spatial'][:, 0]
adata.obs['y_array'] = adata.obsm['spatial'][:, 1] 

In [12]:
x_array = adata.obs["x_array"].tolist()
y_array = adata.obs["y_array"].tolist()
x_pixel = adata.obs["x_pixel"].tolist()
y_pixel = adata.obs["y_pixel"].tolist()

In [13]:
s = 1
b = 49
# adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=s, histology=True)
adj = spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)

Calculateing adj matrix using xy only...


In [ ]:
## adj.csv 可以不保存，有些占空间

In [16]:
# np.savetxt('../data/adj.csv', adj, delimiter=',')

In [17]:
adata.var_names_make_unique()
spg.prefilter_genes(adata, min_cells=3) # avoiding all genes are zeros
spg.prefilter_specialgenes(adata)

In [18]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

In [19]:
## set hyper-parameters
p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)

Run 1: l [0.01, 1000], p [0.0, 33887.555647638044]
Run 2: l [0.01, 500.005], p [0.0, 33290.22265625]
Run 3: l [0.01, 250.0075], p [0.0, 31081.642578125]
Run 4: l [0.01, 125.00874999999999], p [0.0, 24446.515625]
Run 5: l [0.01, 62.509375], p [0.0, 13029.0673828125]
Run 6: l [0.01, 31.2596875], p [0.0, 4620.94970703125]
Run 7: l [0.01, 15.63484375], p [0.0, 1342.0965576171875]
Run 8: l [0.01, 7.822421875], p [0.0, 359.1611633300781]
Run 9: l [0.01, 3.9162109375], p [0.0, 92.30227661132812]
Run 10: l [0.01, 1.9631054687499998], p [0.0, 22.829200744628906]
Run 11: l [0.01, 0.9865527343749999], p [0.0, 5.068014144897461]
Run 12: l [0.01, 0.49827636718749996], p [0.0, 0.6026670932769775]
Run 13: l [0.25413818359374996, 0.49827636718749996], p [0.0017273426055908203, 0.6026670932769775]
Run 14: l [0.37620727539062493, 0.49827636718749996], p [0.1195368766784668, 0.6026670932769775]
Run 15: l [0.4372418212890624, 0.49827636718749996], p [0.3120381832122803, 0.6026670932769775]
Run 16: l [0.46

In [20]:
#For this toy data, we set the number of clusters=7 since this tissue has 7 layers
n_clusters=7
#Set seed
r_seed=t_seed=n_seed=100
#Search for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7


2023-04-07 07:53:08.993191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 07:53:09.503688: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-07 07:53:11.218983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.8/site-packages/cv2/../../lib64:
2023-04-07 07:53:11.219235: W t

Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  7
recommended res =  0.7


In [21]:
## Run SpaGCN
clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Do cluster refinement(optional)
#shape="hexagon" for Visium data, "square" for ST data.
adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refined_pred"]=refined_pred
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
#Save results
adata.write_h5ad("../GF_SpaGCN_results.h5ad")

Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch  50
Epoch  60
Epoch  70
Epoch  80
Epoch  90
Epoch  100
Epoch  110
Epoch  120
Epoch  130
Epoch  140
Epoch  150
Epoch  160
Epoch  170
Epoch  180
Epoch  190


In [23]:
set(adata.obs['pred'])

{0, 1, 2, 3, 4, 5, 6}

In [29]:
set(adata.obs['refined_pred'])

{0, 1, 2, 3, 4, 5, 6}

In [30]:
adata=sc.read("../GF_SpaGCN_results.h5ad")
#adata.obs should contain two columns for x_pixel and y_pixel
#Set colors used
# plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C"]

#Plot spatial domains
domains="pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("../pred.png", dpi=600)
plt.close()

#Plot refined spatial domains
domains="refined_pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("../refined_pred.png", dpi=600)
plt.close()

In [ ]:
def run_spagcn(adata, clu_num, img=None, offset=(0, 0), random_seed=100, p=0.5, normalize=False, filtered=False): # for stereo data which must have spatial imfo in .obsm [optional]:image that map to spatial
    
    import SpaGCN as spg
    import torch 
    import random
    import scanpy as sc
    
    adata.obs["x_pixel"] = adata.obsm['spatial'][:, 0] - offset[0]
    adata.obs["y_pixel"] = adata.obsm['spatial'][:, 1] - offset[1]
    adata.obs['x_array'] = adata.obsm['spatial'][:, 0]
    adata.obs['y_array'] = adata.obsm['spatial'][:, 1] 
    
    x_array = adata.obs["x_array"].tolist()
    y_array = adata.obs["y_array"].tolist()
    x_pixel = adata.obs["x_pixel"].tolist()
    y_pixel = adata.obs["y_pixel"].tolist()
    
        
    #Calculate adjacent matrix
    s = 1
    b = 49
    adj = spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, 
                                   x_pixel=x_pixel, y_pixel=y_pixel, 
                                   image=img, beta=b, alpha=s, 
                                   histology=True) if img \
        else spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
        
    #If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
    # adj = spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
    if not filtered:
        adata.var_names_make_unique()
        spg.prefilter_genes(adata, min_cells=3) # avoiding all genes are zeros
        spg.prefilter_specialgenes(adata)
        
    #Normalize and take log for UMI
    if not normalize:
        sc.pp.normalize_per_cell(adata)
        sc.pp.log1p(adata)
        
    print('**********************************')
    p = p 
    #Find the l value given p
    l = spg.search_l(p, adj, start=0.01, 
                     end=50, tol=0.01, max_run=100)
    
    print('**********************************')
    # If the number of clusters known, we can use the spg.search_res() fnction to search for suitable resolution(optional)
    # eg. set the number of clusters=7 since the tissue has 7 layers
    n_clusters = clu_num
    #Set seed
    r_seed = t_seed = n_seed = random_seed
    #Seaech for suitable resolution
    res = spg.search_res(adata, adj, l, n_clusters, 
                         start=0.7, step=0.1, tol=5e-3, 
                         lr=0.05, max_epochs=20, r_seed=r_seed, 
                         t_seed=t_seed, n_seed=n_seed)
    
    print('**********************************')
    clf = spg.SpaGCN()
    clf.set_l(l)
    #Set seed
    random.seed(r_seed)
    torch.manual_seed(t_seed)
    np.random.seed(n_seed)
    #Run
    clf.train(adata, adj, init_spa=True, init="louvain",
            res=res, tol=5e-3, lr=0.05, max_epochs=300)
    
    y_pred, prob = clf.predict()
    adata.obs[f"pred_{clu_num}"] = y_pred
    adata.obs[f"pred_{clu_num}"] = adata.obs[f"pred_{clu_num}"].astype('category')
    
    print('**********************************')
    #Do cluster refinement(optional)
    #shape="hexagon" for Visium data, "square" for ST data.
    adj_2d = spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
    refined_pred = spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs[f"pred_{clu_num}"].tolist(), dis=adj_2d, shape="hexagon")
    adata.obs[f"refined_pred_{clu_num}"] = refined_pred
    adata.obs[f"refined_pred_{clu_num}"] = adata.obs[f"refined_pred_{clu_num}"].astype('category')
    
    return adata